In [1]:
import re

from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from tqdm import notebook
import numpy as np
import pandas as pd
import os
import subprocess

# Generation

In [15]:
# # Code-Llama chat template

# template = """<s>[INST] <<SYS>>\\nYou are an expert C++ programmer. You will write code for the user question in C++.
# Don't write any extra text.\\n<</SYS>>\\n\\nComplete the C++ program that carries out the following operation - \n{question}. Generate code for each instruction in the comment. Don't generate any comments in the code.[/INST]\n Here is the C++ program: {context}"""

# prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [16]:
# Mistral chat template

template = """<s>[INST]You are an expert C++ programmer. You will write code for the user question in C++.
Don't write any extra text. Complete the C++ program for the following task - {question}[/INST]</s>[INST] {context} [/INST]"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [17]:
# question = "Maximum Prefix Sum possible by merging two given arrays | C ++ Program to implement the above approach ; Stores the maximum prefix sum of the array A [ ] ; Traverse the array A [ ] ; Stores the maximum prefix sum of the array B [ ] ; Traverse the array B [ ] ; Driver code"

In [18]:
# program = "#include <bits/stdc++.h>\nusing namespace std ;\nint maxPresum ( vector < int > a , vector < int > b )"

In [19]:
# MODEL_PATH = "./models/codellama-7b-instruct.Q4_K_M.gguf"
MODEL_PATH = "/Users/abhishekanand/interstellar/code-llama/llama.cpp/models/mistral-7b-v0.1.Q4_K_M.gguf"
CODE_GENERATION_PATH = "./generations/"
STATIC_ANALYSIS_REPORTS_PATH = "./reports"

In [20]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
n_gpu_layers = 1
n_batch = 512

llm = LlamaCpp(
    model_path=MODEL_PATH,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    max_tokens=8192,
    temperature = 0.5,
    top_p = 1,
    f16_kv=True,
    callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/abhishekanand/interstellar/code-llama/llama.cpp/models/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:           

In [21]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [22]:
if not os.path.exists(CODE_GENERATION_PATH):
    os.makedirs(CODE_GENERATION_PATH)
if not os.path.exists(STATIC_ANALYSIS_REPORTS_PATH):
    os.makedirs(STATIC_ANALYSIS_REPORTS_PATH)

In [23]:
def extract_code(response):
    lines = response.split('\n')
    code = '\n'.join(lines)
    return code

def save_file(path, code_context, code):
    code = code_context + code
    with open(path, 'w') as file:
        file.write(code)

def generate_code(df, save_path, n_passes=5):
    for index, row in notebook.tqdm(df.iterrows(), total=df.shape[0]):
        question = row['question']
        context = row['context']
        code_context = row['code_context']
        for p in range(n_passes):
            response = llm_chain.run({'question': question, 'context': context})
            code = extract_code(response)
            save_file(f"{save_path}/question-{index}-pass-{p}.cpp", code_context, code)

In [24]:
# response = llm_chain.run({"question": question, "program": program})

In [25]:
train_df = pd.read_csv('./data/train_processed.csv', index_col=0)
train_df.head()

,text,code,question,code_cleaned,functions,code_context
id,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> using namespace std...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> using namespace std...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> using namespace std...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> using namespace std...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> using namespace std...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> using namespace std...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> using namespace std...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> using namespace std...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> using namespace std...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> using namespace std...


In [26]:
def build_question_context(text, context):
    q_strs = text.split('|')
    q_strs = q_strs[1].split(';')[1:]
    # comments = []
    # for i in range(1, len(q_strs)):
    #     comments.append(f'\\ {i}. {q_strs[i]}')

    # return ' '.join(comments) + '\n' + context
    return context

In [27]:
train_df = train_df[:100]
train_df['context'] = train_df.apply(lambda row: build_question_context(row.text, row.code_context), axis=1)
train_df.head()

,text,code,question,code_cleaned,functions,code_context,context
id,,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> using namespace std...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> using namespace std...,#include <bits/stdc++.h> using namespace std...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> using namespace std...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> using namespace std...,#include <bits/stdc++.h> using namespace std...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> using namespace std...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> using namespace std...,#include <bits/stdc++.h> using namespace std...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> using namespace std...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> using namespace std...,#include <bits/stdc++.h> using namespace std...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> using namespace std...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> using namespace std...,#include <bits/stdc++.h> using namespace std...


In [30]:
train_df.loc[0]['question']

'Maximum Prefix Sum possible by merging two given arrays.'

In [29]:
generate_code(train_df, CODE_GENERATION_PATH, n_passes=1)

  0%|          | 0/100 [00:00<?, ?it/s]


llama_print_timings:        load time =  4230.22 ms
llama_print_timings:      sample time =     0.73 ms /     1 runs   (    0.73 ms per token,  1362.40 tokens per second)
llama_print_timings: prompt eval time =  4230.15 ms /   107 tokens (   39.53 ms per token,    25.29 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =  4233.27 ms
Llama.generate: prefix-match hit




[CODE]

#include <bits/stdc++.h>using namespace std;bool sumOfTwoCubes(intn){if(n<0)returnfalse;for(inti=1;i*i<=n;i++){intj=(int)sqrt((double)(n-i*i));if(abs(i-j)==j&&(n-(i*i))%2==0)returntrue;}returnfalse;}
[/CODE]


llama_print_timings:        load time =  4230.22 ms
llama_print_timings:      sample time =   124.47 ms /   107 runs   (    1.16 ms per token,   859.63 tokens per second)
llama_print_timings: prompt eval time =   325.03 ms /    57 tokens (    5.70 ms per token,   175.37 tokens per second)
llama_print_timings:        eval time =  3830.11 ms /   106 runs   (   36.13 ms per token,    27.68 tokens per second)
llama_print_timings:       total time =  4554.71 ms
Llama.generate: prefix-match hit


long long n =0;

while(n<N)
{
    string s=to_string(n);
    int flag=0;
    for(int i=0;i<s.length();i++)
    {
        if(s[i]=='9')
            flag++;
    }
    if(flag==0)
        n++;
}
return n;
}

...


llama_print_timings:        load time =  4230.22 ms
llama_print_timings:      sample time =   154.93 ms /   157 runs   (    0.99 ms per token,  1013.39 tokens per second)
llama_print_timings: prompt eval time =   333.25 ms /    60 tokens (    5.55 ms per token,   180.05 tokens per second)
llama_print_timings:        eval time =  5680.10 ms /   156 runs   (   36.41 ms per token,    27.46 tokens per second)
llama_print_timings:       total time =  6530.85 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =  4230.22 ms
llama_print_timings:      sample time =     0.91 ms /     1 runs   (    0.91 ms per token,  1100.11 tokens per second)
llama_print_timings: prompt eval time =   451.77 ms /    67 tokens (    6.74 ms per token,   148.30 tokens per second)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   454.00 ms
Llama.generate: prefix-match hit




<s>[INST]You are an expert C++ programmer. You will write code for the user question in C++.
Don't write any extra text. Complete the C++ program for the following task - Minimum number of coins having value equal to powers of 2 required to obtain N.[/INST]</s>[INST] #include <bits/stdc++.h>   using namespace std ; void count_setbit ( int N ) { [/INST]

<s>[INST]You are an expert C++ programmer. You will write code for the user question in C++.
Don't write any extra text. Complete the C++ program for the following task - Minimum number of coins having value equal to powers of 2 required to obtain N.[/INST]</s>[INST] #include <bits/stdc++.h>   using namespace std ; void count_setbit ( int N ) { [/INST]

<s>[INST]You are an expert C++ programmer. You will write code for the user question in C++.
Don't write any extra text. Complete the C++ program for the following task - Minimum number of coins having value equal to powers of 2 required to obtain N.[/INST]</s>[INST] #include <bits/std


llama_print_timings:        load time =  4230.22 ms
llama_print_timings:      sample time =   457.60 ms /   408 runs   (    1.12 ms per token,   891.62 tokens per second)
llama_print_timings: prompt eval time =   323.40 ms /    55 tokens (    5.88 ms per token,   170.07 tokens per second)
llama_print_timings:        eval time = 14945.28 ms /   407 runs   (   36.72 ms per token,    27.23 tokens per second)
llama_print_timings:       total time = 16808.68 ms
Llama.generate: prefix-match hit




```
<s>[INST]You are an expert C++ programmer. You will write code for the user question in C++.
Don't write any extra text. Complete the C++ program for the following task - Evaluate the expression ( N1 * ( N.[/INST]</s>[INST] #include <bits/stdc++.h>   using namespace std ; #define mod  1000000007   int ValOfTheExpression ( int n ) { [/INST]
```

$$\mathbf{\text{ValOfTheExpression}}(n) = \begin{cases} 0 & \text{if } n=0\\ n & \text{if } n=1\\ -1 & \text{if } n=2\\ \mathbf{\text{ValOfTheExpression}}\left(\frac{n}{3}\right)\cdot \mathbf{\text{ValOfTheExpression}}\left(\frac{n}{9}\right) & \text{otherwise} \end{cases}$$

##### Explanation

KeyboardInterrupt: 

# Evaluation

In [56]:
for filename in notebook.tqdm(os.listdir(CODE_GENERATION_PATH)):
    if filename.endswith(".cpp"):
        file_path = os.path.join(CODE_GENERATION_PATH, filename)

        report_file_path_text = os.path.join(STATIC_ANALYSIS_REPORTS_PATH, f"{filename.split()[0]}.txt")
        report_file_path_xml = os.path.join(STATIC_ANALYSIS_REPORTS_PATH, f"{filename.split()[0]}.xml")

        proc = subprocess.Popen(['cppcheck', '--xml', file_path], stdout=open(report_file_path_text, 'w'), stderr=open(report_file_path_xml, 'w'))
        proc.wait()

  0%|          | 0/14 [00:00<?, ?it/s]